<a href="https://colab.research.google.com/github/dottybusch/exercises/blob/main/joins_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### SQL JOINS

Up until now, we have been using SQL to retrieve data from only one table. But it is rare that a database will contain only one table. For various reasons including efficiency, security, functionality, etc. it is inadvertent that data corresponding to a single project/organization/domain will end up being divided into multiple tables.

SQL allows you to reference the data in more than one tables using the rows (records) from the `primary key` from one table with their presence in another table called as `foreign key`.

Think of a scenario where customer data is organized into multiple tables called _PersonalDetails_, _OrderHistory_, _ReturnsHistory_. Each unique customer can be identified by assigning a _CustomerID_ that will stay constant in all the tables.

This referencing to retrieve data from multiple tables is achieved through `JOIN` and `UNION` in SQL.

---
To learn how to use JOINs, first let's set up the data and the SQL environment.

In [1]:
!pip install -q pandas

print("\033[92mPandas installed successfully\033[00m")

Pandas installed successfully


In [2]:
# Import the libraries
import pandas as pd

print("\033[92mPandas library imported successfully\033[00m")

# The URL where the data is located
countries_url = 'https://github.com/FootlooseNFree/my_files/blob/main/countries.csv?raw=true'
emissions_url = 'https://github.com/FootlooseNFree/my_files/blob/main/emissions.csv?raw=true'

# Import the Countries data from the URL
df_countries = pd.read_csv(countries_url)
print('\033[92mCountries data imported successfully\033[00m')

# Import the Emissions data from the URL
df_emissions = pd.read_csv(emissions_url)
print('\033[92mEmissions data imported successfully\033[00m')

Pandas library imported successfully
Countries data imported successfully
Emissions data imported successfully


In [3]:
# Import sqlite3
import sqlite3

# The function runs the SQL query
def run_query():
  con = sqlite3.connect('myDatabase.db')
  cur = con.cursor()

  # Create Countries table and add the data to it
  df_countries.to_sql('Countries', con, if_exists='replace', index=False)
  df_emissions.to_sql('Emissions', con, if_exists='replace', index=False)

  # Run the SQL query
  pd.set_option('display.max_columns', None)
  pd.set_option('display.max_rows', None)

  query = input('Enter your SQL Query:  ')
  try:
    df_output = pd.read_sql_query(query, con)
  except Exception as e:
    print(e)
    print('Try again!!!')
    return None

  con.close()
  return df_output

#### Check the first 5 rows in the `Countries` table

In [ ]:
query_result = run_query()
query_result

Enter your SQL Query:  SELECT * FROM Countries LIMIT 5;


,Country,Population,Capital City,Region
0,Afghanistan,42045000,Kabul,Asia
1,Albania,2363314,Tirana,Europe
2,Algeria,47400000,Algiers,Africa
3,American Samoa,49710,Pago Pago,Oceania
4,Andorra,87682,Andorra la Vella,Europe


#### Check the first 5 rows in the `Emissions` table

In [ ]:
query_result = run_query()
query_result

Enter your SQL Query:  SELECT * FROM Emissions LIMIT 5;


,CountryName,Emission,PerCapita,WorldShare
0,China,"12,667,428,430",8.89,32.88
1,United States,"4,853,780,240",14.21,12.60
2,India,"2,693,034,100",1.89,6.99
3,Russia,"1,909,039,310",13.11,4.96
4,Japan,"1,082,645,430",8.66,2.81


### JOINS

Based on how you want to combine the data from the two tables, `JOIN`s are further classified into different types.\
While combining two table with JOINs, the `first table` is considered as `LEFT` and the `second table` as `RIGHT`.

Second important notation is the column reference. Due to possible conflicts between two tables, it is good practice to specify the table name along with the column name separated by `.`.\
E.g. `table_1.column1, table_2.column1, table_2.column_3,...`

---
#### `INNER JOIN`
The `INNER JOIN` keyword selects rows (records) that have matching values in both tables.

Syntax:
```
SELECT table_n1.column_n1, table_n1.column_n1, ...
FROM table_1
INNER JOIN table_2
ON table_1.column_name = table_2.column_name;

# The column name from table_1 is the primary key that has matching foreign key values in table_2
```
---
### Query 1 (Example)
`Countries` table has the columns `Country`, `Population`, `Capital City`, `Region`

`Emissions` table has the columns `CountryName`, `Emission`, `PerCapita`, `WorldShare`

`Country` is the primary key in `Countries` table that has matching elements in the foreign key column `CountryName` in the `Emissions` table.

**Query:** Join the columns `Population` and `Region` from `Countries` with `Emission` from `Emmisions` table, using the matching elements of `Country` key from each table.
```
SELECT Countries.Country, Countries.Population, Countries.Region, Emissions.Emmision
FROM Countries
INNER JOIN Emissions
ON Countries.Country = Emissions.CountryName;
```

In [ ]:
query_result = run_query()
query_result

### `LEFT JOIN`
The `LEFT JOIN` keyword returns all records from the left table (table1), and the matching records from the right table (table2). If table 2 has no matching entries in the column, it gets the `NULL` value.

Syntax:
```
SELECT table_n1.column_n1, table_n1.column_n1, ...
FROM table_1
LEFT JOIN table_2
ON table_1.column_name = table_2.column_name;
```
### Query 2
Perform a left join for the `Countries` table on the `Emissions` table using the `Country` and `CountryName` as matching keys from the respective tables. Retrieve the columns with country name, population and per capita. Also sort it according to the per capita column in descending order.

**Hint:**
- Pay attention to which column is in which table and use the notation `table_name.column_name`
- Notice the retrieved data if there are any `NULL` values.

In [ ]:
query_result = run_query()
query_result

### `RIGHT JOIN`
The `RIGHT JOIN` works in the opposite way as the LEFT JOIN. The keyword returns all records from the right table (table2), and the matching records from the left table (table1). `NULL` values will be shown for missing matches.

Syntax:
```
SELECT table_n1.column_n1, table_n1.column_n1, ...
FROM table_1
RIGHT JOIN table_2
ON table_1.column_name = table_2.column_name;
```

In [ ]:
query_result = run_query()
query_result

### `FULL JOIN` or `FULL OUTER JOIN`
The `FULL OUTER JOIN` keyword returns all records when there is a match in left (table1) or right (table2) table records.

Syntax:
```
SELECT table_n1.column_n1, table_n1.column_n1, ...
FROM table_1
FULL OUTER JOIN table_2
ON table_1.column_name = table_2.column_name;
```

In [ ]:
query_result = run_query()
query_result

### `UNION` and `UNION ALL`
The `UNION` operator is used to combine the result-set of two or more `SELECT` statements.

- Every `SELECT` statement within UNION must have the same number of columns
- The columns must also have similar data types
- The columns in every `SELECT` statement must also be in the same order

The `UNION` operator selects only distinct values by default. To allow duplicate values, use `UNION ALL`

Syntax:
```
SELECT column_name(s) FROM table_1
UNION / UNION ALL
SELECT column_name(s) FROM table_2
```

In [ ]:
query_result = run_query()
query_result